In [1]:
import numpy as np
import time

In [2]:
with open("input_20.txt", "r") as fh:
    grid = np.array([list(l.strip()) for l in fh.readlines()])

In [3]:
def pretty_print(grid, remove_negative=False):
    for r in grid:
        for c in r:
            if remove_negative and c < 0:
                c = ""
            print(c, end="\t")
        print()
    print()

def all_adjacent(loc):
    all_locs = [(loc[0]+1, loc[1]), (loc[0]-1, loc[1]), (loc[0], loc[1]+1), (loc[0], loc[1]-1)]
    return [l for l in all_locs if l[0]<grid.shape[0] and l[1] <grid.shape[1] and l[0]>=0 and l[1] >= 0] # last and added for 20b

def navigate_grid(grid):
    # Walk through starting point grid, with explicit assumption that there is only one path
    grid_out = -np.ones(grid.shape)
    next_locs = {}
    cur_loc = tuple(i.item() for i in np.where(grid=="S"))
    time = 0
    grid_out[cur_loc] = time
    while grid[cur_loc] != "E":
        time += 1
        new_locs_found = 0
        for next_step in all_adjacent(cur_loc):
            if (grid[next_step] == "." or grid[next_step] == "E") and grid_out[next_step] == -1:
                new_locs_found += 1
                grid_out[next_step] = time
                next_locs[cur_loc] = next_step
                cur_loc = next_step
                break
    return grid_out, next_locs

times, next_locs = navigate_grid(grid)

In [4]:
# 20a
savings_threshold = 100
cur_loc = tuple(i.item() for i in np.where(times==0))
savings = {}
while cur_loc in next_locs:
    for next_step in all_adjacent(cur_loc):
        if next_step in next_locs:
            continue
        for second_step in all_adjacent(next_step):
            s = times[second_step] - times[cur_loc] - 2
            if s >= savings_threshold:
                if s not in savings:
                    savings[s] = 1
                else:
                    savings[s] += 1
        
    cur_loc = next_locs[cur_loc]
print("20a:", sum(savings.values()))

20a: 1463


In [5]:
# 20b
t0 = time.time()
# Loc db is required to avoid multiple np.where calls
loc_db = {}
for num in range(0, int(np.max(times)) + 1):
    loc_db[num] = tuple(i.item() for i in np.where(times==num))

# Find all possible shortcuts/cheats
num_cheats =0
for num in range(0, int(np.max(times))):
    loc = loc_db[num]
    for cheat_num in range(num+1, int(np.max(times))+1):
        cheat_loc = loc_db[cheat_num]
        # Check minimum distance and calculate cheat
        dist = abs(loc[0]-cheat_loc[0]) + abs(loc[1]-cheat_loc[1])
        if dist <= 20 and cheat_num-num-dist >= savings_threshold:
            num_cheats += 1

print("20b:", num_cheats, f"in {time.time()-t0} sec.")

20b: 985332 in 6.098456144332886 sec.
